In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, losses, callbacks

2024-03-20 20:22:07.355142: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 20:22:07.355372: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 20:22:07.461034: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-20 20:22:07.676705: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-20 20:22:09.016355: W tensorflow/compiler/tf2

In [2]:
MODEL_NAME = "primeiro_treino"
PASTA_FOURIER = "data/fourier/"
SPS = 200

In [3]:
df = pd.read_csv('data/scores.csv')

In [4]:
df = df.sample(frac=1, random_state=200)

In [5]:
df_val = df.head(2700)
df_train = df.tail( df.shape[0]- 2700 )

In [189]:
def get_data(eeg):
    return np.load(PASTA_FOURIER+str(eeg)+".npy")


def get_start_end(row, pos, max_size):
    if pos == len(row['janelas']):
        return max_size, max_size

    j = row['janelas'].split(',')[0].split('-')
    return float(j[0])*2, float(j[1])*2

def score_data(row):
    data = get_data(row['eeg_id'])
    max_size = data.shape[0]
    scores = [] 
    j_pos = 0
    start, end = get_start_end(row, j_pos, max_size)

    for i in range(max_size):
        if end < i:
            j_pos += 1
            start, end = get_start_end(row, j_pos, max_size)

        if i >= start and i <= end:
            scores.append( [row['seizure_vote'],row['lpd_vote'],row['gpd_vote'],row['lrda_vote'],row['grda_vote'],row['other_vote'] ] )
        else:
            scores.append( [0.0, 0.0, 0.0, 0.0, 0.0, 0.0] )


    return data, scores


In [190]:
def get_scored_data():
    for i, row in df_train.iterrows():
        yield( score_data(row) )


In [191]:
model = models.Sequential()
model.add(layers.Conv2D(1, (2, 2), activation='relu', input_shape=(20, 200, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(60, activation='relu'))
model.add(layers.Dense(6, activation='sigmoid'))


In [192]:
model.summary()

Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_41 (Conv2D)          (None, 19, 199, 1)        5         
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 9, 99, 1)          0         
 ng2D)                                                           
                                                                 
 flatten_34 (Flatten)        (None, 891)               0         
                                                                 
 dense_64 (Dense)            (None, 60)                53520     
                                                                 
 dense_65 (Dense)            (None, 6)                 366       
                                                                 
Total params: 53891 (210.51 KB)
Trainable params: 53891 (210.51 KB)
Non-trainable params: 0 (0.00 Byte)
_______________

In [193]:
next(get_scored_data())[1].shape

AttributeError: 'list' object has no attribute 'shape'

In [201]:
model.compile(
    optimizer = 'adam',
    loss=tf.keras.losses.CategoricalCrossentropy()
)

In [202]:
train_data = tf.data.Dataset.from_generator(
        get_scored_data,
        output_signature = (
            tf.TensorSpec(shape=(None, 20, 200), dtype=tf.float16),
            tf.TensorSpec(shape=(None, 6), dtype=tf.float16)
        )
    )

In [203]:

history = model.fit(
    train_data,
    # epochs=10,
    # batch_size=10,
    # validation_split=0.2,
    verbose= 1,
    callbacks = [
        callbacks.EarlyStopping(monitor="val_loss", patience=2),
        callbacks.ModelCheckpoint(MODEL_NAME, save_best_only = True)
    ]
)

2024-03-20 21:43:08.633333: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f898c0136b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-20 21:43:08.633373: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Quadro M1000M, Compute Capability 5.0
2024-03-20 21:43:08.657827: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1710967388.808040    5471 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


    656/Unknown - 48s 71ms/step - loss: 1.2443

In [ ]:
history